In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/YLP

/content/drive/MyDrive/YLP


In [ ]:
# Unless mentioned, code is from:
# Adversarial Example Generation, Inkawhich, N. and Uriegas, E.,  2018, Available from: https://github.com/pytorch/tutorials/blob/master/beginner_source/fgsm_tutorial.py , Accessed: 20 Feb 2022

total 96
drwx------ 2 root root  4096 Dec 13 10:50 data/
drwx------ 3 root root  4096 Dec 13 09:16 modules/
drwx------ 2 root root  4096 Dec 13 09:21 __pycache__/
-rw------- 1 root root    49 Dec 13 09:38 requirements2.txt
-rw------- 1 root root    92 Dec  2 09:15 requirements.txt
drwx------ 2 root root  4096 Dec 13 10:56 results/
-rw------- 1 root root 24681 Mar 14 07:17 RKNet.py
drwx------ 2 root root  4096 Mar  9 09:55 runs/
-rw------- 1 root root 51501 Mar 14 07:17 utils.py


In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
from RKNet import RKNet

# NOTE: This is a hack to get around "User-agent" limitations when downloading MNIST datasets
#       see, https://github.com/pytorch/vision/issues/3497 for more information
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)


In [ ]:
## JB Code Block ##
epsilons = [2**(-7), 2**(-6), 2**(-5), 2**(-4), 2**(-3), 2**(-2), 2**(-1)]

#filename for trained parabolic network
pretrained_model = os.path.join('results',"current_pdenet.pt")
use_cuda=True

In [ ]:
## JB Code Block ##

shift = transforms.Normalize(
    mean=[0.5,0.5,0.5],
    std=[1.,1.,1.],
  )



# CIFAR-10 Test dataset and dataloader declaration
# ResNet accepts images of size 224 x 224, so size all images to 256 x 256 and then crop
test_data =  datasets.CIFAR10('../data', train=False, download=True, transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    shift]))


test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

Files already downloaded and verified
CUDA Available:  True


In [ ]:
## JB Code Block ##

# Filename for trained ResNet network
PATH = os.path.join('results','current_resnet_20220319_0711.pt')

# Modify output layer so that dimensions match saved model
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(512,10)

# Load saved weights
model.load_state_dict(torch.load(PATH))
# Set to eval mode - turns dropout layers into identity
model.eval()

# Load model to GPU
model = model.to(device)

In [ ]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    if epsilon == 0:
      return image
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, -0.5, 0.5)
    # Return the perturbed image
    return perturbed_image

In [ ]:
def test( model, device, test_loader, epsilon ):

    # Accuracy counter
    correct = 0
    adv_examples = []
    
    # JB code line: keep track of incorrect examples as they are not attacked
    incorrect_init = 0
  
    # Loop over all examples in test set
    for data, target in test_loader:
        
        data, target = data.to(device), target.to(device)


        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
          incorrect_init += 1
          continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if epsilon == 0 and final_pred.item() != init_pred.item():
          print("Mismatch!")
          print(target.item(),init_pred.item(),final_pred.item())
        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    fgsm_tested = len(test_loader)-incorrect_init
    
    # JB code line: accuracy is only out of examples that were FGSM tested
    final_acc = correct/float(fgsm_tested)
    # Original line: final_acc= correct/float(len(test_loader))
    
    # JB code line: match print-out with changes in line above
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, fgsm_tested, final_acc))
    # print("Epsilon: {}\tInitial Accuracy = {} / {} = {}".format(epsilon, incorrect_init, len(test_loader), init_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [ ]:
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model, device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

Epsilon: 0	Test Accuracy = 9356 / 9356 = 1.0
Epsilon: 0.0078125	Test Accuracy = 1528 / 9356 = 0.1633176571184267
Epsilon: 0.015625	Test Accuracy = 2431 / 9356 = 0.25983326207781104
Epsilon: 0.03125	Test Accuracy = 2983 / 9356 = 0.3188328345446772
Epsilon: 0.0625	Test Accuracy = 2147 / 9356 = 0.2294784095767422
Epsilon: 0.125	Test Accuracy = 1353 / 9356 = 0.14461308251389482
Epsilon: 0.25	Test Accuracy = 1019 / 9356 = 0.1089140658401026
Epsilon: 0.5	Test Accuracy = 1084 / 9356 = 0.115861479264643
